# 0. Loading

First we activate the `examples` environment and then we load a couple of packages

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using Revise, PtyLab, Parameters, EllipsisNotation, Colors
using ImageShow, ImageCore, FFTW, BenchmarkTools, ImageIO, Downloads, HDF5, Plots, ColorSchemes
using IndexFunArrays
FFTW.set_num_threads(4)

  Activating project at `~/.julia/dev/PtyLab.jl/examples`
Precompiling project...
  ✓ PtyLab
  1 dependency successfully precompiled in 8 seconds. 328 already precompiled.


# 1. Data Download

It is ~350MB large, so please be patient for a few moments (or minutes).
The file is stored temporarily somewhere else. So if you restart this notebook, it downloads it again.
However, you can also manually specify the file path down below.

In [ ]:
dataset = Downloads.download("https://figshare.com/ndownloader/files/38419391", verbose=true)

* Couldn't find host figshare.com in the (nil) file; using defaults
*   Trying 34.253.18.52:443...
* Connected to figshare.com (34.253.18.52) port 443 (#0)
* mbedTLS: Connecting to figshare.com:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: offers h2
* ALPN: offers http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 06:84:57:DC:03:6E:5E:FD:D5:17:8C:39:07:4B:9B:A7
* issuer name       : C=US, O=Amazon, CN=Amazon RSA 2048 M02
* subject name      : CN=*.figshare.com
* issued  on        : 2023-02-10 00:00:00
* expires on        : 2023-04-26 23:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.figshare.com
*     dNSName : figshare.com
* key usage         : Digital Signature, Key Encipherment
* ext key usage     : TLS Web Server Authentication, TLS Web Client Authentication
* certificate 

# 2. Load into ExperimentalDataCPM

In [ ]:
datah5 = h5open(dataset, "r")

In [ ]:
read(datah5, "dxd")

In [ ]:
read(datah5, "zo")# .* read(datah5, "zo") ./ (240 .* read(datah5, "dxd")) * 240

In [ ]:
(-reconstruction.Np/2:reconstruction.Np/2-1)*reconstruction.dxp

In [ ]:
reconstruction.xp

In [ ]:
experimentalData = ExperimentalDataCPM(dataset);

In [ ]:
simshow(experimentalData.ptychogram[:, :, 1], γ=0.3)

In [ ]:
simshow(experimentalData.ptychogram[:, :, 1001], γ=0.3)

In [ ]:
experimentalData.zo

# 3. Load into ReconstructionCPM

In [ ]:
reconstruction = ReconstructionCPM(experimentalData, cuda=true);

In [ ]:
reconstruction.encoder = reconstruction.encoder'

In [ ]:
reconstruction.encoder .-= mean(reconstruction.encoder, dims=(2,))

# 4. Choose engine

In [ ]:
params = Params(fftshiftFlag = false, 
                 transposePtychogram = false, 
                 comStabilizationSwitch = true,
                 randPositionOrder = true)

engine = PtyLab.ePIE(betaProbe = 0.75f0, betaObject = 0.75f0, numIterations = 10)

In [ ]:
reconstruction.npsm = 2

In [ ]:
reconstruction.entrancePupilDiameter = reconstruction.Np/3 * reconstruction.dxp

# 5. Reconstruct

In [ ]:
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);

In [ ]:
#reconstruction.probe .*= randn(size(reconstruction.probe)) ;
#reconstruction.probe .*= cis.(rr2(size(reconstruction.probe), scale=0.12));

In [ ]:
@time p, o = PtyLab.reconstruct(engine, params, reconstruction);

In [ ]:
simshow(Array(reconstruction.object[:, :, 1,1,1,1]))

In [ ]:
[simshow(Array(reconstruction.probe[:, :, 1,1,1,1])) simshow(Array(reconstruction.probe[:, :, 1,1,2,1]))]